# EDA Project - Airbnb Paris

## Background

When traveling to Europe and booking an Airbnb rental property for a vacation, I try to spend less on lodging to be able to afford good dining and tours.  For Paris specifically, I find that the options close to the centre, that is close to the top attractions i.e. Eiffel tower, Champs Elysees are pricier, and most of the properties are unavailable as well, so I end up staying further from the centre, and when I do, I am always worried that I might have booked a property that is way too far out from the centre of the city, and or far from a metro station -- since actual location is not provided until after you have made the reservation, or unless you contact host and ask.  Distance to the station is relative depending on how far one is willing to walk, but for me, more than half a mile walk to a station is considered far, since I feel I have to save all my energy for the leisurely walking one does around in Paris.

## Questions for Exploration / Goals


1) to be able to explore and answer the following questions:

* How does location influence property rental price, reviews and availability?  

* What other features drive the price of an airbnb rental property?

  i.e. price vs location, price vs distance from a top attraction, etc.



2) recommend Airbnb properties given a certain criteria, and enable a more informed decision for the traveler: 

* walking distance to one of the top attractions (within 2 miles)
* walking distance to metro station (within 1 mile)
* rating >= 9

* other user defined criteria i.e. 
* number of beds
* number of bedrooms
* price range
* type of property

## Datasets

The Airbnb dataset was downloaded from the OpenDataSoft website below.  The data was filtered to only extract and download data pertaining to Paris, France Airbnb rental properties.

[Airbnb Paris Dataset](https://public.opendatasoft.com/explore/dataset/airbnb-listings/table/?disjunctive.host_verifications&disjunctive.amenities&disjunctive.features&refine.city=Paris&dataChart=eyJxdWVyaWVzIjpbeyJjaGFydHMiOlt7InR5cGUiOiJjb2x1bW4iLCJmdW5jIjoiQ09VTlQiLCJ5QXhpcyI6Imhvc3RfbGlzdGluZ3NfY291bnQiLCJzY2llbnRpZmljRGlzcGxheSI6dHJ1ZSwiY29sb3IiOiJyYW5nZS1jdXN0b20ifV0sInhBeGlzIjoiY2l0eSIsIm1heHBvaW50cyI6IiIsInRpbWVzY2FsZSI6IiIsInNvcnQiOiIiLCJzZXJpZXNCcmVha2Rvd24iOiJyb29tX3R5cGUiLCJjb25maWciOnsiZGF0YXNldCI6ImFpcmJuYi1saXN0aW5ncyIsIm9wdGlvbnMiOnsiZGlzanVuY3RpdmUuaG9zdF92ZXJpZmljYXRpb25zIjp0cnVlLCJkaXNqdW5jdGl2ZS5hbWVuaXRpZXMiOnRydWUsImRpc2p1bmN0aXZlLmZlYXR1cmVzIjp0cnVlfX19XSwidGltZXNjYWxlIjoiIiwiZGlzcGxheUxlZ2VuZCI6dHJ1ZX0%3D)

Paris RER/metro dataset was downloaded from the website below and will be used for further exploration:


[Accessibilite des gares et stations de Metro](https://data.ratp.fr/explore/?sort=modified)


### Limitations
* the Airbnb data from OpenDataSoft was last updated in 2017, availability information will be based on what has been captured from the last update.
* Historical data for the property prices and availabity are not available, so we are unable to check for variability of price and availability on different seasons/time.
* For computing distances Haversine formula was used, which would be a good estimate for computing the distance between two points, but this distance could be different from the actual walking distance.

## Cleaning the Data

The geojson file format was read and used for this analysis. The original download file has eighty-six (86) columns, columns which are not needed for analysis, i.e. calendar_last_scraped, calendar_updated, listing_url, etc. were dropped to minimize columns and focus on cleaning and analysis of data which will be useful this EDA.

The 'city' column was dropped from the file as well, since all the records downloaded are those pertaining to Airbnb rental properties in Paris, France.

Further research on zipcodes was done, and found that zipcodes 75001 - 75020 correspond to Arrondissements 1 through 20 in Paris.  Since analysis will be focused on properties within the centre of Paris, the records with zipcodes outside of this range were deleted.

*More information on Paris Arrondissements in wikipedia: https://en.wikipedia.org/wiki/Arrondissements_of_Paris

### Missing Values:
Missing values in beds and bedrooms were imputed using the following rules:
* If 'beds' has a missing value, and 'bedrooms' has a valid value, 'beds' is set with the value of 'bedrooms'.
* If both 'bedrooms' and 'beds' are missing, both fields are set to 1, which is the average number of beds and bedrooms,
and it would be safe to assume that if a property is being rented out in Airbnb, that there is at least one bed, and we will count that as one bedroom, regardless if the property is a studio apartment, where there is no pyhsical division between the rooms.

There were several records as well with missing 'scores' value, a new column 'rating_ind' was created to flag rated = 1, versus unrated = 0 records, so further analysis can be done between these two populations.  The record was flagged as rated, if all the score values have been populated, and set to unrated, if at least one of the score values has not been populated.

### Data Transformation and Feature Engineering:

The following fields were added to the dataset as well:

<br>1) arrondissement - this was derived from the last two digits of the zipcode

2) arrond_name - arrondissement name was populated and compared with the values populated in the neighbourhood_cleansed field.  There were some differences found, since no information is available on how the neighbourhood_cleansed field was derived, for the sake of consistency with the categorizing by Arrondissement, the arrond_name will be used for analysis instead of the neighbourhood_cleansed field.  


3) A column for each of the Distances between the Airbnb property and each of the [2018 top 10 attractions in Paris](https://www.tripadvisor.com/Attractions-g187147-Activities-Paris_Ile_de_France.html#ATTRACTION_SORT_WRAPPER) were derived as well using the [Haversine forumla](https://community.esri.com/groups/coordinate-reference-systems/blog/2017/10/05/haversine-formula), and these features were used to analyze and determine relationship with the rental property price.

["Musee d’Orsay", "Sainte-Chapelle", "Palais Garnier - Opera", "Notre Dame Cathedral", "Musee de l’Orangerie", "Luxembourg Gardens", "Louvre", "Eiffel", "Pont Alexandre III", "Le Marais"]

4) close_to_attraction - this is an indicator if the property is within 2 miles of one of the top 10 attractions

5) closest_attraction - this is the attraction closest to the Airbnb property

6) attraction_dist - distance between the Airbnb property and the closest attraction.




## Exploring the Data

In [2]:
import pandas as pd
dfParis = pd.read_pickle('airbnb_paris/airbnb_Paris_updt_0606.p')

Plotting the Airbnb rental property prices out shows us that the price of Airbnb rental properties in Paris  has a right-skewed unimodal distribution.  We can see from the graph below it shows that majority of the properties are rented out are below \$200, with the average below \$100 which is at \$93.72, with the median only at $75.  We can see that there are some outliers -- properties over \$400, causing a huge difference between the mean and the median price, and for the distribution to be right skewed.

[graph 1.0]
<img src="files/images/airbnb_price_dist.png">

In [3]:
dfParis.price.describe()

count    52295.000000
mean        93.718673
std         71.128178
min          0.000000
25%         55.000000
50%         75.000000
75%        105.000000
max        999.000000
Name: price, dtype: float64

The plot below shows the correlation of the other features with the Airbnb rental property price:
* It does make sense that the distance between the top attractions and the Airbnb property is negatively correlated with the price, and so is with the Arrondissement, and 'close_to_attraction' has a positive correlation with the 'price'.

* With regards to distance between Le Marais and the Airbnb property being positively correlated, it is possible that those properties which are farther from Le Marais are closer to the other top attractions, giving us that inconsistent correlation as far as distances from top attractions are concerned.  For analysis, the 'close_to_attraction' indicator together with the 'attraction_dist' will be used, instead of the individual distances to the top attraction.

* As far as the other property features are concerned, we can see that the number of beds, bedrooms, and accomodations are positively correlated with the Airbnb property price.  It is expected that the bigger the accomodation is, the rental price will be higher as well.

[graph 2.0]
<img src="files/images/features_corr.png">

#### Exploring Location (Arrondissements) and Availability_30

If we look at each of our top 10 attractions in Paris, we can see that the attractions are in arrondissements 1 through 9.

As we saw from the correlation graph above --  Arrondissement values are negatively correlated with the price, that is, the Airbnb properties in the lower Arrondissements on average costs more than the properties in the higher arrondissements, which is what the graph below shows.  Although it appears that rental prices in Arr 8 - Champs Elysees costs the most, and Arr 9 - Opera has a lower rental prices compared to arrondissements 1 - 8.  This could be because Champs Elysees is generally a pricier neighborhood in Paris, and it is a very famous avenue in Paris.

<img src="files/images/df_arrondissements.png">



[graph 3.0]
<img src="files/images/line_pricebyArr.png">

With regards to the availability, the graphs below show that the price does have direct relationship on the availability of the Airbnb rental properties in Paris. Properties which are available for 5 or more days in the next 30 days, has an average price around \$100 or more.

[graph 4.0]
<img src="files/images/airbnb_price_byAvail.png">

If we look at availability of Airbnb rental properties by price range, we can see that for properties within \$100/night budget the options for available properties are much less in Arrondissements 1 through 8, increasing your budget will give you better chances of getting a property close to the attractions (in Arrondissements 1 through 8), and if that is not an option, renting outside of arrondissments 1 through 8 will give one a better option of renting out a property within the \$100/night budget.

[graph 5.0]
<img src="files/images/Avail_less100.png">

#### Exploring Rating Indicator with Availability and Price

The graphs above shows all properties, that is for both rated properties, and unrated which includes properties with incomplete scores.  If we break down the availability of properties, we can see that we have higher availability for 'unrated' properties, which is what we are expecting, since people tend to stay in places with higher ratings, where other renters have had a good experience staying.

[graph 6.0]
<img src="files/images/box_rating_avail.png">

With regards to the price, there appears to be no significant difference between rental price for rated properties versus the unrated ones.  As we have seen in the Correlation table above in [graph 2.0] review_scores fields: [review_scores_value, review_scores_communication, review_scores_location, review_scores_cleanliness, review_scores_checkin, review_scores_accuracy fields] did show a positive correlation with the price, but not a strong correlation. The strongest correlation was with review_scores_location, which makes sense because as we have seen locations closer to the attractions (Arr 1-8) do cost more compared to properties outside of Arrondissements 1-8.

[graph 7.0]
<img src="files/images/box_ratingprice.png">

### Exploring other Features

#### Exploring Property Type

In the graphs below, we can see that the property type does affect the price of the property, as in the case of Camper/RV, Igloo, and villa property types.  We can see that average price for these properties are around the \$300 and above price range, which is way above the average and median price of the rental properties in Paris.  These property types represent less than 1% of the total Airbnb rental properties in Paris, thus causing the skewness we have observed in the price distribution.

<img src="files/images/airbnb_price_byPtype.png">
<img src="files/images/property_type.png">



#### Exploring the distance to a Top Attraction with Property price and Availability

After filtering the dataset down further to get closer to the recommendations:

The dataset was filtered to only include Airbnb properties which are within 2 miles of one of the top attractions,
AND within 1 mile of a RER/metro station.  Finding that most of the properties are within 2 miles of a top attraction, and a mile from a station, the dataset was then classified further to get number of Attractions within 1 mile of the Airbnb property, and number of stations within 0.2 miles of the property.


We can see that the properties which are not within a mile of any of the top attraction has the lowest median price, those which have at least one attraction within 1 mile (1 to 4 attractions), median price are pretty close, and those which are within a mile to more than five (5) attractions, have the higher median price.

It is the same for properties and its closeness to a station.  Those properties which are beyond 0.2 miles to a station has the lowest median price.  Those with at least one station within 0.2 miles (1-2 stations) median price are pretty close to each other, and those with 3-4 stations within 0.2 miles have the higher priced properties.

<img src="files/images/box_recmd_price_vs_sitecount.png">

<img src="files/images/box_recmd_price_vs_stncount.png">

## Summary

### 1) How does location influence property rental price, reviews and availability?  

* Based on our findings from this EDA, we confirmed that the properties in Arrondissements 1 through 8, where most of the top attractions are located, cost more than properties outside of those arrondissements.

* And also, availability of properties in Arrondissements 1 - 8 are lower especially for properties rented out less than \$100/night, compared to properties outside of Arrondissements 1 - 8

* With regards to the rated properties, and unrated properties, availability of unrated properties are higher compared to the rated properties.  An option for a traveler would be to consider the unrated properties to have more options for booking closer to the centre.



1b) What other features drive the price of an airbnb rental property?

* We did see that the number of accomodation/beds/bedrooms has a strong positive correlation with property rental price

* the location of the property i.e. Arrondissement, (close_to_attraction == 1) that is if Airbnb property is within 2 miles of one of the top 10 attractions

* the property type does affect the price of the Airbnb rental property, and in this case, is causing the skewness in the price distribution, because at least 95% of the Airbnb rental properties in Paris has a property type = Apartment, and those properties with the higher cost \$300 and above, such as the Camper/RV, Igloo, and the villa only represents less than 1% of the total Airbnb rental properties in Paris.

* after getting the distances from our top 10 attractions, the data does show that properties outside of a mile from these attractions cost less, and average price of properties which are within a mile to one or more of the attractions would cost more.

* with regards to the distance from a station, the data does show that average price of properties which are within 0.2 miles to at least one station costs more than those properties which are more than 0.2 miles to a station.




### 2) Airbnb Paris Recommendations




Recommendation filter 1:

* properties which are within 1 mile at least one of the top attractions

* Room type == Entire home/apt

* property is within 0.2 miles more than 2 stations


Recommendation filter 2: (if #1 filter results to less than 20 properties, results from #2 will be added to the recommendation list)

* properties which are within 1 mile at least one of the top attractions AND
* Room type == Entire home/apt AND
* property is within 0.2 miles at least 2 stations

Below are Recommended Airbnb properties with the base criteria below, and additional filters applied as defined
in Recommendation filters 1 and 2:  
* Beds: >= 1  
* Accomodates >= 2  
* Available_30 >=  5 days  
* Price Max: $85  
* Rating Ind = 1 (rated properties only)
* Room type = Entire home/Apt
* Review_Scores_Rating = 100.0
   

In [8]:
import pandas as pd
results = pd.read_pickle('airbnb_paris/recommendation_results_0615.p')
results.reset_index(drop = True, inplace = True)
results.style

Paris Map with recommendations can be viewed below:

[Paris_map](https://cdn.rawgit.com/ayeshavm/K2_Project2_EDA/8de36452/images/paris_map.html)

## Further Research and Analysis



* Refine recommendation to assign weights to features. i.e. give each arrondissement a weight, each attraction a weight, etc.

* Incorporate additional information i.e. restaurants and cafes close to the area, and include that in analysis and recommendation process as well.

* Using more current data via Airbnb APIs.

* Text analysis on comments and other features/amenities.